Table A3: Perceptions of change in the level of alcohol related issues in the local area in the last 12 months by demographics (%)

In [1]:
from databaker.framework import *
import pandas as pd 

In [2]:
import requests
from cachecontrol import CacheControl
from cachecontrol.caches.file_cache import FileCache
from cachecontrol.heuristics import LastModified
from pathlib import Path

session = CacheControl(requests.Session(),
                       cache=FileCache('.cache'),
                       heuristic=LastModified())

sourceFolder = Path('in')
sourceFolder.mkdir(exist_ok=True)

inputURL = 'https://www.justice-ni.gov.uk/sites/default/files/publications/justice/ni-omnibus-survey-oct-2016-alcohol-drugs-tabular.xlsx'
inputFile = sourceFolder / 'ni-omnibus-survey-oct-2016-alcohol-drugs-tabular.xlsx'
Value = session.get(inputURL)
with open(inputFile, 'wb') as f:
  f.write(Value.content)

https://www.justice-ni.gov.uk/sites/default/files/publications/justice/ni-omnibus-survey-oct-2016-alcohol-drugs-tabular.xlsx

In [3]:
tab = loadxlstabs(inputFile, sheetids='Table A3')[0]

Loading in/ni-omnibus-survey-oct-2016-alcohol-drugs-tabular.xlsx which has size 191159 bytes
Table names: ['Table A3']


In [4]:
# select obs. Remove anything down-right from F3 as there's some kruft
observations = tab.excel_ref('B5').expand(DOWN).expand(RIGHT).is_not_blank() - tab.excel_ref("F3").expand(DOWN).expand(RIGHT)

In [5]:
observations

{<B28 0.094>, <E37 40.0>, <D8 'n<100'>, <B51 0.076>, <E48 559.0>, <Q36 'Total'>, <D39 0.048>, <B58 0.032>, <B43 0.087>, <D37 'n<100'>, <E12 260.0>, <D43 0.121>, <B37 'n<100'>, <E36 408.0>, <C8 'n<100'>, <C58 0.897>, <D51 0.132>, <E16 465.0>, <E15 416.0>, <D31 0.081>, <D36 0.089>, <B5 0.07>, <D25 0.085>, <B27 0.046>, <E44 278.0>, <B25 0.044>, <D48 0.099>, <C24 0.763>, <C10 0.832>, <E42 238.0>, <E5 881.0>, <C12 0.836>, <E57 561.0>, <E8 57.0>, <C20 0.85>, <E51 230.0>, <D27 0.063>, <B8 'n<100'>, <C37 'n<100'>, <D24 0.122>, <B44 0.053>, <E43 365.0>, <B57 0.094>, <D16 0.104>, <C36 0.857>, <E21 13.0>, <D10 0.108>, <E58 320.0>, <C35 0.755>, <B19 0.072>, <D9 'n<100'>, <E20 428.0>, <E56 304.0>, <C47 0.83>, <D28 0.161>, <C42 0.847>, <B32 0.075>, <C32 0.807>, <C9 'n<100'>, <D11 0.091>, <C44 0.851>, <E28 144.0>, <B20 0.062>, <E19 337.0>, <C31 0.857>, <C26 0.814>, <C52 0.834>, <C25 0.871>, <C51 0.792>, <C28 0.745>, <C27 0.891>, <B39 0.065>, <E55 257.0>, <D47 0.115>, <D12 0.093>, <C43 0.792>, <B10 0.

In [6]:
Alcoholissue = tab.excel_ref('B3').expand(RIGHT).is_not_blank()
Alcoholissue

{<B3 'Better'>, <D3 'Worse'>, <C3 'About the same'>, <E3 'Unweighted base'>}

In [7]:
sect = tab.excel_ref('A5').expand(DOWN).is_not_blank()
sect

{<A15 'Male'>, <A46 'Dependants'>, <A24 'Belfast'>, <A9 '25-34'>, <A12 '65 and over'>, <A62 '1. Results exclude "don\'t know" and refusals.'>, <A19 'Catholic'>, <A35 'Single, that is never married'>, <A20 'Protestant'>, <A25 'Northern'>, <A10 '35-49'>, <A28 'Western'>, <A55 'Belfast'>, <A56 'Urban, excluding Belfast'>, <A41 'Level of qualifications'>, <A31 'In paid employment'>, <A26 'South Eastern'>, <A57 'All urban'>, <A14 'Gender'>, <A39 'Widowed'>, <A61 'Source: Northern Ireland Omnibus Survey, October 2016'>, <A52 'No disability / illness'>, <A16 'Female'>, <A23 'Health and Social Care Trust'>, <A32 'Not in paid employment'>, <A30 'Employment status'>, <A43 'Secondary'>, <A58 'Rural'>, <A51 'Has disability / illness'>, <A44 'Tertiary'>, <A11 '50-64'>, <A42 'Primary'>, <A48 'No dependants'>, <A5 'All adults'>, <A54 'Area type'>, <A7 'Age of household reference person (HRP)1'>, <A27 'Southern'>, <A37 'Married and separated from husband / wife'>, <A50 'Disability / illness'>, <A18 'R

In [8]:
category = tab.excel_ref('A').expand(DOWN).by_index([7,14,18,23,30,34,41,46,50,54])
category

{<A46 'Dependants'>, <A54 'Area type'>, <A41 'Level of qualifications'>, <A7 'Age of household reference person (HRP)1'>, <A23 'Health and Social Care Trust'>, <A30 'Employment status'>, <A50 'Disability / illness'>, <A18 'Religion'>, <A34 'Marital status'>, <A14 'Gender'>}

In [9]:
Dimensions = [
            HDim(Alcoholissue,'Alcohol related issue response',DIRECTLY,ABOVE),
            HDim(category,'Category',CLOSEST,ABOVE),
            HDim(sect,'Sect',DIRECTLY,LEFT),
            HDimConst('Measure Type', 'Percent'),
            HDimConst('Unit','People'),
            ]

In [10]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
# savepreviewhtml(c1)

In [11]:
new_table = c1.topandas()
new_table

NoLookupError: No header found for <Q25 'Total'>

In [ ]:
new_table.columns = ['Value' if x=='OBS' else x for x in new_table.columns]

In [ ]:
new_table['Value'] = pd.to_numeric(new_table['Value'], errors='coerce')

In [ ]:
new_table = new_table[new_table['Value'].notnull()]

In [ ]:
new_table['Value'] = new_table['Value'].apply(str)

In [ ]:
def user_perc(x,y):
    
    if str(x) == 'Unweighted base':
        return 'Count'
    else:
        return y
    
new_table['Measure Type'] = new_table.apply(lambda row: user_perc(row['Alcohol related issue response'],row['Measure Type']), axis = 1)

In [ ]:
new_table['Category'] = new_table['Category'].str.rstrip('1')

In [ ]:
new_table['Demographic1'] = new_table['Category'].fillna('') + ' - ' + new_table['Sect'].fillna('')

In [ ]:
new_table['Demographic1'] = new_table['Demographic1'].str.lstrip(' - ')

In [ ]:
new_table = new_table[['Demographic1','Alcohol related issue response','Measure Type','Value','Unit']]

In [ ]:
new_table.head(5)

In [ ]:
import urllib.request as request
import csv
r = request.urlopen('https://raw.githubusercontent.com/ONS-OpenData/ref_alcohol/master/codelists/demographic.csv').read().decode('utf8').split("\n")
reader = csv.reader(r)
for line in reader:
    print(line)

In [ ]:
import io
import requests
url="https://raw.githubusercontent.com/ONS-OpenData/ref_alcohol/master/codelists/demographic.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

In [ ]:
Final_table = pd.merge(new_table, c, how = 'left', left_on = 'Demographic1', right_on = 'Label')

In [ ]:
Final_table.columns = ['Demographic' if x=='Notation' else x for x in Final_table.columns]

In [ ]:
Final_table['Alcohol related issue response'] = Final_table['Alcohol related issue response'].map(
    lambda x: {
        'Better' : 'better', 
        'About the same' : 'same',
        'Worse': 'worse' ,
        'Unweighted base': 'unweighted-base'
        }.get(x, x))

In [ ]:
Final_table = Final_table[Final_table['Alcohol related issue response'].notnull()]

In [ ]:
Final_table = Final_table[['Demographic','Alcohol related issue response','Measure Type','Value','Unit']]

In [ ]:
destinationFolder = Path('out')
destinationFolder.mkdir(exist_ok=True, parents=True)

Final_table.to_csv(destinationFolder / ('table3.csv'), index = False)

In [ ]:
Final_table['Alcohol related issue response'].unique()